In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import warnings 
import pickle
import pylab 
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer


In [2]:

warnings.filterwarnings('ignore')

dataset=pd.read_csv('dataset.csv',compression='gzip')

st.subheader('BMI Calculator')

weight_in_kgs =1

height_in_mtr =1

age=1

2023-06-17 10:19:19.298 
  command:

    streamlit run c:\Users\anshul\miniconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [3]:

#sex=male (1) or female(2)

sex=1

BMI = (weight_in_kgs)/((height_in_mtr)*(height_in_mtr))
print(BMI)


1.0


In [4]:

if BMI <16:
    print('under weight')
elif BMI <17 and BMI >=16:
    print('under weight')
elif BMI <18.5 and BMI >=17:
    print('under weight')
elif BMI <25 and BMI >=18.5:
    print('normal')
elif BMI <30 and BMI >=25:
    print('over weight')
else :
    print('obesity')

under weight


In [5]:
if sex == 1:
    BMR = 10 * weight_in_kgs + 6.25 *height_in_mtr*1000 - 5 *age  + 5
    print('kcal req per day {}'.format(BMR))
else :
    BMR = 10 * weight_in_kgs + 6.25 *height_in_mtr*1000 - 5 *age  -161
    print('kcal req per day {}'.format(BMR))

kcal req per day 6260.0


In [6]:
number_of_meals=3

In [7]:
if number_of_meals==3:
    meals_calories_perc={'breakfast':0.35,'lunch':0.40,'dinner':0.25}
    print('breakfast:{},lunch:{},dinner:{}'.format(0.35*BMR,0.4*BMR,0.25*BMR))
    
    Calories=0.4*BMR
elif number_of_meals==4:
    meals_calories_perc={'breakfast':0.30,'morning snack':0.05,'lunch':0.40,'dinner':0.25}
    print('breakfast {}, morning snack {},lunch {},dinner {}'.format(0.30*BMR,0.05*BMR,0.40*BMR,0.25*BMR))
    #for lunch only
    Calories=0.40*BMR
else:
    meals_calories_perc={'breakfast':0.30,'morning snack':0.05,'lunch':0.40,'afternoon snack':0.05,'dinner':0.20}
    print('breakfast {}, morning snack {},lunch {},afternoon snack {},dinner {}'.format(0.30*BMR,0.05*BMR,0.40*BMR,0.05*BMR,0.20*BMR))
    #for lunch only
    Calories=0.40*BMR

print('for lunch only')
print(Calories)

breakfast:2191.0,lunch:2504.0,dinner:1565.0
for lunch only
2504.0


In [8]:
FatContent = 10
SaturatedFatContent = 10
CholesterolContent = 10
SodiumContent= 10
CarbohydrateContent= 10
FiberContent= 10
SugarContent= 10
ProteinContent= 10

df=[Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent]

df=np.array(df,dtype='int').reshape(-1, 1)
df=df.reshape(1,9)
print(df)

[[2504   10   10   10   10   10   10   10   10]]


In [9]:
neigh = pickle.load(open('neigh.pkl', 'rb'))

In [10]:

def recommand(dataframe,_input,max_nutritional_values,ingredient_filter=None,params={'return_distance':False}):
    extracted_data=extract_data(dataframe,ingredient_filter,max_nutritional_values)
    prep_data,scaler=scaling(extracted_data)
    neigh=nn_predictor(prep_data)
    pipeline=build_pipeline(neigh,scaler,params)
    return apply_pipeline(pipeline,_input,extracted_data)


def extract_data(dataframe,ingredient_filter,max_nutritional_values):
    extracted_data=dataframe.copy()
    for column,maximum in zip(extracted_data.columns[6:15],max_nutritional_values):
        extracted_data=extracted_data[extracted_data[column]<maximum]
    if ingredient_filter!=None:
        for ingredient in ingredient_filter:
            extracted_data=extracted_data[extracted_data['RecipeIngredientParts'].str.contains(ingredient,regex=False)] 
    return extracted_data

def scaling(dataframe):
    scaler=StandardScaler()
    prep_data=scaler.fit_transform(dataframe.iloc[:,6:15].to_numpy())
    return prep_data,scaler

def nn_predictor(prep_data):
    neigh = NearestNeighbors(metric='cosine',algorithm='brute')
    neigh.fit(prep_data)
    return neigh


def build_pipeline(neigh,scaler,params):
    transformer = FunctionTransformer(neigh.kneighbors,kw_args=params)
    pipeline=Pipeline([('std_scaler',scaler),('NN',transformer)])
    return pipeline

def apply_pipeline(pipeline,_input,extracted_data):
    return extracted_data.iloc[pipeline.transform(_input)[0]]


In [11]:
recommand(dataset,df,Calories)

TypeError: 'float' object is not iterable